In [3]:
key = "D4"
L = LieAlgebra(QQ, cartan_type=key)
R = RootSystem(key)
lattice =R.root_lattice()
space = R.root_space()
alpha = space.basis()
roots= space.roots()
pos_roots = space.positive_real_roots()
short_roots = space.short_roots()
long_roots = space.long_roots()
E = L.basis()
alphacheck = space.alphacheck()
chev_basis = space.roots()+[ alphacheck[i] for i in [1,2] ]
hvee = sum(space.highest_root().associated_coroot().coefficients())+1
n=len(alpha)
V=VectorSpace(QQ,3*len(pos_roots))
basis_vectors = V.basis()

In [4]:
def mult3gen(index,i,j):
    return 1/2*basis_vectors[index+i]+1/2*basis_vectors[index+j];

def mult(index1,index2):
    if index1>index2:
        return mult(index2,index1);
    
    if index1 == index2:
        return basis_vectors[index1];
    
    root1 = pos_roots[floor(index1/3)];
    x1 = Integer(mod(index1,3));
    root2 = pos_roots[floor(index2/3)];
    x2 = Integer(mod(index2,3));
    if root1 == root2:
        return mult3gen(3*floor(index1/3),x1,x2);
    if root1.scalar(root2.associated_coroot()) == 0:
        return V.zero_vector();
    elif root1+root2 in pos_roots:
        sum_index = pos_roots.index(root1+root2);
        if x1==0:
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index+x2];
        elif (x1==1 and x2 == 0):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index+1];
        elif (x1==1 and x2 == 1):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*(2*basis_vectors[3*sum_index+1] - basis_vectors[3*sum_index]);
        elif (x1==1 and x2 == 2):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*(basis_vectors[3*sum_index+1]-basis_vectors[3*sum_index]+basis_vectors[3*sum_index+2]);
        elif (x1==2 and x2 == 0):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index+2];
        elif (x1==2 and x2 == 1):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*(-basis_vectors[3*sum_index]+basis_vectors[3*sum_index+1]+basis_vectors[3*sum_index+2]);
        elif (x1==2 and x2 == 2):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*(2*basis_vectors[3*sum_index+2]-basis_vectors[3*sum_index] );
    elif root2-root1 in pos_roots:
        sum_index = pos_roots.index(root2-root1);
        if x1==0:
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*basis_vectors[3*sum_index+x2];
        elif (x1==1 and x2 == 0):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*(2*basis_vectors[3*sum_index]-basis_vectors[3*sum_index+1]);
        elif (x1==1 and x2 == 1):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2]  -1/4*basis_vectors[3*sum_index];
        elif (x1==1 and x2 == 2):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2]-1/4*(basis_vectors[3*sum_index]-basis_vectors[3*sum_index+1]+basis_vectors[3*sum_index+2]);
        elif (x1==2 and x2 == 0):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*(2*basis_vectors[3*sum_index]-basis_vectors[3*sum_index+2]);
        elif (x1==2 and x2 == 1):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2] -1/4*(basis_vectors[3*sum_index]+basis_vectors[3*sum_index+1]-basis_vectors[3*sum_index+2]);
        elif (x1==2 and x2 == 2):
            return 1/4*basis_vectors[index1] + 1/4*basis_vectors[index2]-1/4*basis_vectors[3*sum_index];
            
        
def multmat(index):
    return Matrix([ mult(index,i) for i in [0..len(basis_vectors)-1] ]);


structure_constants = [ multmat(i) for i in [0..len(basis_vectors)-1] ];
A = FiniteDimensionalAlgebra(QQ,structure_constants)

In [5]:
def CheckEigenvalues(A,a):
    output = true;
    for i in range(A.dimension()):
        b= A.basis()[i];
        output = output and (1/2*(a*b) +(a*(a*(a*b))) == 3/2*(a*(a*b)));
    return output;

def CheckZeroZero(A,a):
    output = true;
    for i in range(A.dimension()):
        for j in range(A.dimension()):
            b= A.basis()[i];
            c= A.basis()[j];
            b_new = a*(a*b) -3/2*(a*b) + 1/2*b;
            c_new = a*(a*c) -3/2*(a*c) + 1/2*c;
            output = output and ( a*(b_new*c_new) == 0*(b_new*c_new) );
    return output;


def CheckHalfHalf(A,a):
    output = true;
    for i in range(A.dimension()):
        for j in range(A.dimension()):
            b= A.basis()[i];
            c= A.basis()[j];
            b_new = a*(a*b) -(a*b) ;
            c_new = a*(a*c) -(a*c);
            output = output and  ( a*(a*(b_new*c_new)) == a*(b_new*c_new) );
    return output;


def CheckZeroHalf(A,a):
    output = true;
    for i in range(A.dimension()):
        for j in range(A.dimension()):
            b= A.basis()[i];
            c= A.basis()[j];
            b_new = a*(a*b) -(a*b) ;
            c_new = a*(a*c) -3/2*(a*c) + 1/2*c;
        output = output and ( a*(b_new*c_new) == 1/2*(b_new*c_new) );
    return output;

def CheckJordanfusionlaw(A,a):
    output = (a*a == a);
    output = output and CheckEigenvalues(A,a);
    output = output and  CheckHalfHalf(A,a);
    output = output and CheckZeroHalf(A,a);
    output = output and CheckZeroZero(A,a);
    return output;

def IsAxial(A):
    n=A.dimension();
    output = true;
    i =0;
    while i < 4 and output:
            output = output and CheckJordanfusionlaw(A,A.basis()[i]);
            i+=1;
    return output;

def IsLikelyJordan(A):
    output = true;
    n = A.dimension();
    for i in [1..10]:
        x = A.random_element();
        y = A.random_element();
        output = output and (x*y)*(x*x) == x*(y*(x*x)) ;
    return output;

print(IsAxial(A))
print(IsLikelyJordan(A))

True
False


In [4]:
M = multmat(0)
basis = M.eigenspaces_left()[2][1].basis()
for i in range(len(basis)):
    print(A(basis[i]))

e0 - 2*e29 - 2*e32
e3 + e12 - e29 - e32
e4 + e13 - e29 - e32
e5 + e14 - e29 - e32
e6
e7
e8
e9
e10
e11
e15 + e21 - e29 - e32
e16 + e22 - e29 - e32
e17 + e23 - e29 - e32
e18 + e24 - e29 - e32
e19 + e25 - e29 - e32
e20 + e26 - e29 - e32
e27 - e29 + e30 - e32
e28 - e29 + e31 - e32
e33
e34
e35


In [5]:
basis10 = M.eigenspaces_left()[1][1].basis() + M.eigenspaces_left()[0][1].basis()
for i in range(len(basis10)):
    print(A(basis10[i]))

e0 - e2
e1 - e2
e3 - e12
e4 - e13
e5 - e14
e15 - e21
e16 - e22
e17 - e23
e18 - e24
e19 - e25
e20 - e26
e27 - e30
e28 - e31
e29 - e32
e0


In [6]:
for i in range(len(basis)):
    for j in range(i,len(basis)):
        print(i," ",j," ",A.0*(A(basis[i])*A(basis[j])))

0   0   0
0   1   0
0   2   0
0   3   0
0   4   0
0   5   0
0   6   0
0   7   0
0   8   0
0   9   0
0   10   0
0   11   0
0   12   0
0   13   0
0   14   0
0   15   0
0   16   0
0   17   0
0   18   0
0   19   0
0   20   0
1   1   0
1   2   0
1   3   0
1   4   0
1   5   0
1   6   0
1   7   0
1   8   0
1   9   0
1   10   0
1   11   0
1   12   0
1   13   0
1   14   0
1   15   0
1   16   0
1   17   0
1   18   0
1   19   0
1   20   0
2   2   0
2   3   0
2   4   0
2   5   0
2   6   0
2   7   0
2   8   0
2   9   0
2   10   0
2   11   0
2   12   0
2   13   0
2   14   0
2   15   0
2   16   0
2   17   0
2   18   0
2   19   0
2   20   0
3   3   0
3   4   0
3   5   0
3   6   0
3   7   0
3   8   0
3   9   0
3   10   0
3   11   0
3   12   0
3   13   0
3   14   0
3   15   0
3   16   0
3   17   0
3   18   0
3   19   0
3   20   0
4   4   0
4   5   0
4   6   0
4   7   0
4   8   0
4   9   0
4   10   0
4   11   0
4   12   0
4   13   0
4   14   0
4   15   0
4   16   0
4   17   0
4   18   0
4   19   0
4   20

In [7]:
A.2*A.4

1/4*e2 + 1/4*e4 + 1/4*e12 - 1/4*e13 - 1/4*e14